__Importing all the Required modules__

In [2]:
#Importing all the necessary modules
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

__Helper Function 1 :: *grayscale(Input Image)* --> Grayscaling is required for preprocessing the image for further mathematical and logical operations. To avoid selecting manual thresholds, OTSU thresholding method is used, which in a nutshell will automaticallly select the thresholds using the input image and mathematical calculations.__

In [3]:
#Converting the Input image to graycale for further processing
#Since selecting threshold is difficult for an image stream for uncontrolled environment images, OTSU threholding method
#is used to automatically select thresholds for the image.
def grayscale(img):
    rows    = img.shape [1]
    cols    = img.shape [0]
    ch      = img.shape [2]   
    
    grayscale_main            = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret,grayscale_output      = cv2.threshold(grayscale_main, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return grayscale_output

__Helper Function 2 :: *canny(Input Image)* --> Canny function from openCV library automatically converts the input image into an edge detected image output for easier classification in further steps.__

In [4]:
#Using openCV methods to directly to transform the input image
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

__Helper Function 3 :: *gaussian_blur(Input Image, Kernel Size)* --> Gaussian Blur is used to voluntarily remove noise signals from the input image, this is better than averaging  filter, where the data of the significant elements can get lost__

In [5]:
#Gaussian blur method is a better way of averaging the pixels in the image for selecting feature in an image
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

__Helper Function 4 :: *region_of_interest(Input Image, Vertices)* --> Based on the image, the regions wherein the object of significance can be expected and taken for calculations instead of checking all the pixel of the input images.__


In [6]:
#To select the ROI image where we expect the lanes or our object of interest to remain prominantly.
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)     
    
    if len(img.shape) > 2:
        channel_count     = img.shape[2]  
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image          = cv2.bitwise_and(img, mask)
    
    return masked_image

__Helper Function 5 :: draw_lines(Image, lines, color, thickness) --> Using the detected line coordinates from the Hough Transform, this function applies the basic line equation to the detected lines and draws lines of user selected color and thickness values.__

In [7]:
#Once lanes coordinates are estimated, this function draws lines using the universal slope equation.
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    #Setting Value to Global Variable
    Image_Center_Threshold = img.shape[1] / 2

    RightLane_X_Points = []
    RightLane_Y_Points = []
    
    LeftLane_X_Points  = []
    LeftLane_Y_Points  = []
    
    RightLane_SlopeIntercepts = []
    LeftLane_SlopeIntercepts  = []
    
    Averaged_RightLane_SlopeIntercepts = []
    Averaged_LeftLane_SlopeIntercepts  = []
    
    for line in lines:        
        for x1,y1,x2,y2 in line:        
           #For Right Lanes           
           if(x1  > Image_Center_Threshold): #Image_Center_Threshold and x2 > Image_Center_Threshold ):
               RightLane_SlopeIntercepts.append(np.polyfit((x1,x2),(y1,y2),1))
               RightLane_X_Points.append([x1,x2])
               RightLane_Y_Points.append([y1,y2])
           #For Left lanes
           else:
               LeftLane_SlopeIntercepts.append(np.polyfit((x1,x2),(y1,y2),1))
               LeftLane_X_Points.append([x1,x2])
               LeftLane_Y_Points.append([y1,y2])
               
    Averaged_RightLane_SlopeIntercepts = np.mean(RightLane_SlopeIntercepts, axis = 0)
    Averaged_LeftLane_SlopeIntercepts  = np.mean(LeftLane_SlopeIntercepts, axis = 0)
    Left_Lane_points  = []
    Right_Lane_points = [] 
        
    # Y Coordinate Calculations
    Right_Y_TopCoordinate    = (max(RightLane_Y_Points))
    Right_Y_BottomCoordinate = (min(RightLane_Y_Points))
    Left_Y_TopCoordinate     = (max(LeftLane_Y_Points))
    Left_Y_BottomCoordinate  = (min(LeftLane_Y_Points))
    #Fixing Common Y Coordinates for the Final Overlay Lines
    Y_TopCoordinate     = max(Right_Y_TopCoordinate[0]      ,Left_Y_TopCoordinate[0])
    Y_BottomCoordinate  = max(Right_Y_BottomCoordinate[0],Left_Y_BottomCoordinate[0])
    
    #Right Lane Bottom Point   
    Right_X_TopCoordinate = int((Y_TopCoordinate - Averaged_RightLane_SlopeIntercepts[1]) / Averaged_RightLane_SlopeIntercepts[0])
    Right_Lane_points.append(Right_X_TopCoordinate)
    Right_Lane_points.append(Y_TopCoordinate)  
    
    #Right Lane Top Point
    Right_X_BottomCoordinate = int((Y_BottomCoordinate - Averaged_RightLane_SlopeIntercepts[1]) / Averaged_RightLane_SlopeIntercepts[0])
    Right_Lane_points.append(Right_X_BottomCoordinate)
    Right_Lane_points.append(Y_BottomCoordinate)    
    
    #Left Lane Bottom Point
    Left_X_TopCoordinate     = int((Y_TopCoordinate - Averaged_LeftLane_SlopeIntercepts[1]) / Averaged_LeftLane_SlopeIntercepts[0])
    Left_Lane_points.append(Left_X_TopCoordinate)
    Left_Lane_points.append(Y_TopCoordinate)

    #Left Lane Top Points
    Left_X_BottomCoordinate  = int((Y_BottomCoordinate - Averaged_LeftLane_SlopeIntercepts[1]) / Averaged_LeftLane_SlopeIntercepts[0])
    Left_Lane_points.append(Left_X_BottomCoordinate)
    Left_Lane_points.append(Y_BottomCoordinate)
    
    draw_output = cv2.line(img, (Right_Lane_points[0] , Right_Lane_points[1]) , (Right_Lane_points[2],Right_Lane_points[3]) , color , thickness)    
    draw_output = cv2.line(img, (Left_Lane_points[0]  , Left_Lane_points[1])  , (Left_Lane_points[2] ,Left_Lane_points[3])  , color , thickness)

    return draw_output

__Helper Function 6 :: hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap) --> This function takes the pre-processed images with the lane marking highlighted/focused, and by using the various tuning thresholds selects the pixels which are a line nature whilst ignoring the scattered the point detections in the preprocessed image.__

In [8]:
#Used to represent images in terms of lines
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines       = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img    = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    hough_image = draw_lines(line_img, lines, color=[255, 0, 0], thickness=10)
    
    return hough_image

__Helper Function 7 :: weighted_img(img, initial_img, α, β, γ) --> Superimposes the line which is detected in thorugh the pipe onto to the image stream, the superimposition parameters can be tuned usingultiplication factors α, β and addition constant γ.__

In [9]:
#Used to superimpose the lane marking that pipeline detects to the original image, or basically the annotated image output 
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):    
    return cv2.addWeighted(initial_img, α, img, β, γ)

__Test Function to the Check the pipeline is working.__

In [10]:
#Test Function to test whether the pipeline works
def main():
    main_image   = cv2.imread('exit-ramp.jpg')

    process_image(main_image)

__The Final Pipeline function which is used to run the detection algorithm__

In [11]:
#Basically the entire pipeline using all the helper functions
def process_image(main_image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    
    gray_image         = grayscale(main_image)
    canny_image        = canny(gray_image, 140, 170)   
    gaussianblur_image = gaussian_blur(canny_image,5)
    
    rows = gaussianblur_image.shape[1]
    cols = gaussianblur_image.shape[0]
    
    top_left_corner   = [rows*(4.5/10) , cols*(3/5)]
    top_right_corner  = [rows*(6/10)   , cols*(3/5)]
    down_left_corner  = [0   , cols]
    down_right_corner = [rows , cols]
    roi_points = np.array( [[top_left_corner,top_right_corner,down_right_corner,down_left_corner]], dtype=np.int32 )
    
    roi_image = region_of_interest(gaussianblur_image,roi_points)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 45  #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    line_image = np.copy(main_image)*0 # creating a blank to draw lines on
    
    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    hough_image = hough_lines(roi_image, rho, theta, threshold,min_line_length, max_line_gap)
    
    final_image = weighted_img(hough_image,main_image,.5,1,0)
    
    return final_image


__Import the Video Processing modules__

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

__These lines take the video in the directory and run the software pipeline on a small section of it which can be manually set in the subclip(X,X).__

In [13]:
    white_output = 'test_videos_output/solidWhiteRight.mp4'
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(5,10)
    clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:17<00:00, 11.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 7.64 s, sys: 245 ms, total: 7.88 s
Wall time: 19.4 s
